In [1]:
import sys
# TO CHANGE
BASEDIR = "../../../.."
sys.path.insert(0, BASEDIR)

In [2]:
from pprint import pprint

In [3]:
from src.kg_model import KnowledgeGraphModel, KnowledgeGraphModelConfig
from src.db_drivers.vector_driver import EmbedderModelConfig
from src.pipelines.memorize import MemPipelineConfig, MemPipeline, LLMExtractorConfig, LLMUpdatorConfig
from src.utils.agent_stat_analyzer import AgentStatAnalyzerConfig
from src.config import DEFAULT_PERSONALAI_KVCACHE_CONFIG

/home/dzigen/Desktop/Projects/PersonalAI/.pai_venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Пример работы с Memorize-пайплайном (по построению графа знаний)

1. Инициализация модели графа знаний

In [4]:
kg_config = KnowledgeGraphModelConfig(
    nodestree_config=None # Модель дерева вершин строиться не будет
)

EMBEDDER_MODEL_PATH = '../../../../models/intfloat/multilingual-e5-small' # PATH TO APPROPRIATE EMBEDDER-MODEL
kg_config.embedders_configs['m-e5-small'] = EmbedderModelConfig(model_name_or_path=EMBEDDER_MODEL_PATH)


kg_model = KnowledgeGraphModel(kg_config)

No sentence-transformers model found with name ../../../../models/intfloat/multilingual-e5-small. Creating a new one with mean pooling.


2. Инициализация Memorize-пайплайна

In [5]:
mem_config = MemPipelineConfig(
    extractor_config=LLMExtractorConfig(lang='ru'),
    updator_config=LLMUpdatorConfig(
        lang='ru', 
        delete_obsolete_info=False # Выключаем механизм по поиску/удалению устревших знаний в графе при добавлении новой информации
    )
)

In [6]:
mem_pipeline = MemPipeline(kg_model, mem_config, DEFAULT_PERSONALAI_KVCACHE_CONFIG, AgentStatAnalyzerConfig())

In [7]:
kg_model.count_items()

{'graph_info': {'triplets': 0, 'nodes': 0},
 'embeddings_info': {'nodes': 0, 'triplets': 0},
 'nodestree_info': None}

In [8]:
print("mem tgen cache:")
pprint(mem_pipeline.get_agent_tgen_stat())
print("mem casual cache:")
pprint(mem_pipeline.get_cache_stat())

mem tgen cache:
{'extractor': {'thesises_extraction_solver': {'generated_tokens_amount': {'count': 0,
                                                                          'count_not_null': 0,
                                                                          'max': None,
                                                                          'mean': None,
                                                                          'median': None,
                                                                          'min': None,
                                                                          'std': None,
                                                                          'sum': None},
                                              'inference_elapsed_time': {'count': 0,
                                                                         'count_not_null': 0,
                                                                         'max': None,
           

3. Формируем триплеты (из входных текстовых фрагментов на естественном языке) и добавляем в граф знаний

In [10]:
TEXTS = [
    "Шла саша по шоссе.",
    "Шла маша по шоссе.",
    "Теплоход плыл по водному каналу.",
    "Моторная лодка плыла по реке."
]


In [15]:
for i, text in enumerate(TEXTS):
    print(f"\n{i+1}.Входной текст: ", text)
    extracted_triplets, rinfo = mem_pipeline.remember(text)
    print("Извлечённые из текста триплеты:")
    pprint(extracted_triplets, width=200)
    print("Информация о завершении операции: ", rinfo)



1.Входной текст:  Шла саша по шоссе.
Извлечённые из текста триплеты:
[Triplet(start_node=Node(name='саша', type=<NodeType.object: 'object'>, prop={}, stringified='саша', id='72adc4fc2f53b1344fb5b56d6ed81b1f'),
         relation=Relation(name='проходит', type=<RelationType.simple: 'simple'>, prop={}, id='da0b488d41d6458245fac501fc2e5a0b'),
         end_node=Node(name='шоссе', type=<NodeType.object: 'object'>, prop={}, stringified='шоссе', id='956c2d5e231f655cee06efa6a32969e9'),
         stringified='саша проходит шоссе',
         id='017d461c6c38d3260ca906ac65953228'),
 Triplet(start_node=Node(name='саша', type=<NodeType.object: 'object'>, prop={}, stringified='саша', id='72adc4fc2f53b1344fb5b56d6ed81b1f'),
         relation=Relation(name='hyper', type=<RelationType.hyper: 'hyper'>, prop={}, id='b7cb6abc49e154002f814107ef31a20f'),
         end_node=Node(name='1. саша проходит по шоссе', type=<NodeType.hyper: 'hyper'>, prop={}, stringified='1. саша проходит по шоссе', id='b7cb6abc49e154

In [16]:
kg_model.count_items(detailed=True)

{'graph_info': {'triplets': {'simple': 4, 'hyper': 14, 'episodic': 20},
  'nodes': {'object': 14, 'hyper': 5, 'episodic': 4}},
 'embeddings_info': {'nodes': {'object': {'nodes_dense': 14,
    'nodes_sparse_bm25': 14},
   'hyper': {'nodes_dense': 5, 'nodes_sparse_bm25': 5},
   'episodic': {'nodes_dense': 4, 'nodes_sparse_bm25': 4}},
  'triplets': {'triplets_dense': 13, 'triplets_sparse_bm25': 13}},
 'nodestree_info': None}

In [13]:
kg_model.check_consistency()

True

In [18]:
print("mem tgen cache:")
pprint(mem_pipeline.get_agent_tgen_stat())
print("mem casual cache:")
pprint(mem_pipeline.get_cache_stat())

mem tgen cache:
{'extractor': {'thesises_extraction_solver': {'generated_tokens_amount': {'count': 0,
                                                                          'count_not_null': 0,
                                                                          'max': None,
                                                                          'mean': None,
                                                                          'median': None,
                                                                          'min': None,
                                                                          'std': None,
                                                                          'sum': None},
                                              'inference_elapsed_time': {'count': 0,
                                                                         'count_not_null': 0,
                                                                         'max': None,
           

In [17]:
mem_pipeline.clear_agent_tgen_stat()
mem_pipeline.clear_kv_caches()

In [19]:
del mem_pipeline

In [15]:
kg_model.clear()

In [16]:
kg_model.count_items()

{'graph_info': {'triplets': 0, 'nodes': 0},
 'embeddings_info': {'nodes': 0, 'triplets': 0},
 'nodestree_info': None}

In [20]:
del kg_model